# US Stock Screening (Post Market) v0

In this notebook we try to achieve
- Screen US Common Stock with previous day Volume
- Send Telegram notice

Timeline as follow:

|| T0 | T1 | T2 | T3 |
| --- | --- | --- | --- | --- |
| Expected trend | x | Volume Rise | Volume Drop | Price Rise |
| Action | x | **Create Potential Stocks List** | Enter Market near day end | |

Import Library

In [ ]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from scipy import stats

Import Token

In [ ]:
# from secrets import IEX_SANDBOX_API_TOKEN as IEX_CLOUD_API_TOKEN
# baseUrl = f'https://sandbox.iexapis.com/stable'
# SEND_TELEGRAM = False

from secrets import IEX_CLOUD_API_TOKEN
baseUrl = f'https://cloud.iexapis.com/stable'
SEND_TELEGRAM = True

IEX Cloud API test call

In [ ]:
symbol = 'AMZN'
symbolString = 'AAPL,TSLA,BIAF,AMZN'

# quote of a stock
batchQuotePreviousUrl = f'/stock/market/batch?symbols={symbolString}&types=quote,previous&token={IEX_CLOUD_API_TOKEN}'

apiUrl = baseUrl + batchQuotePreviousUrl
# data = requests.get(apiUrl)
# print(data.status_code)
# print(data.text)
# data = requests.get(apiUrl).json()
# data

## Preparation

Fetch all available symbol, filter with "US" and "common stock".

In [ ]:
refDataUrl = f'/ref-data/symbols?token={IEX_CLOUD_API_TOKEN}'
apiUrl = baseUrl + refDataUrl

data = requests.get(apiUrl).json()
# data

In [ ]:
allSymbolsDataFrame = pd.json_normalize(data)

In [ ]:
allUsSymbolsDataFrame = allSymbolsDataFrame[
    (allSymbolsDataFrame['type'] == 'cs') &
    (allSymbolsDataFrame['region'] == 'US')
]
allUsSymbolsDataFrame

Define chunks to prepare for batch API call

In [ ]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
symbolGroups = list(chunks(allUsSymbolsDataFrame['symbol'], 100))
symbolStrings = []
for i in range(0, len(symbolGroups)):
    symbolStrings.append(','.join(symbolGroups[i]))
#     print(symbol_strings[i])

symbolStrings

## Data Fetching

Create main DataFrame, including volume of today, previous trading date.

In order to get the previous trading date, we need to call **/previous**, or else we can use **/quote/previousVolume** directly

Data imported from IEX Cloud API.

In [ ]:
%%time

potentialDataFrameColumns = [
    'Ticker',
    'Price',
    '30 day Average Volume',
    'T0 Date',
    'T0 Volume',
    'T1 Volume',
    'T1 Volume Ratio',
]

potentialDataFrame = pd.DataFrame(columns = potentialDataFrameColumns)

for symbolString in symbolStrings:
    batchApiUrlCall = baseUrl + f'/stock/market/batch?symbols={symbolString}&types=quote,previous&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batchApiUrlCall).json()
#     print(data)
#     print(data.status_code)
    for symbol in symbolString.split(','):
        try:
            latestPrice = data[symbol]['quote']['latestPrice']
        except KeyError:
            latestPrice = np.NaN
        try:
            avgTotalVolume = data[symbol]['quote']['avgTotalVolume']
        except KeyError:
            avgTotalVolume = np.NaN
        try:
            latestVolume = data[symbol]['quote']['latestVolume']
        except KeyError:
            latestVolume = np.NaN
            
        try:
            t0Date = data[symbol]['previous']['date']
        except (KeyError, TypeError):
            t0Date = np.NaN
        try:
            t0Volume = data[symbol]['previous']['volume']
        except (KeyError, TypeError):
            t0Volume = np.NaN
            
        potentialDataFrame = potentialDataFrame.append(
            pd.Series(
                [
                    symbol,
                    latestPrice,
                    avgTotalVolume,
                    t0Date,
                    t0Volume,
                    latestVolume,
                    'N/A'
                ],
                index = potentialDataFrameColumns
            ),
            ignore_index = True
        )

In [ ]:
potentialDataFrame

## Data Cleaning

Clean out data with None from API

In [ ]:
potentialDataFrame[potentialDataFrame.isnull().any(axis = 1)]
# potentialDataFrame[potentialDataFrame['Price'].isnull()]

In [ ]:
potentialDataFrame = potentialDataFrame.dropna()
potentialDataFrame

Drop data with 0 Volume

In [ ]:
potentialDataFrame[
    (potentialDataFrame['T0 Volume'] == 0) |
    (potentialDataFrame['T1 Volume'] == 0)
]

In [ ]:
potentialDataFrame.drop(
    potentialDataFrame[
        (potentialDataFrame['T0 Volume'] == 0) |
        (potentialDataFrame['T1 Volume'] == 0)
    ].index,
    inplace = True
)
potentialDataFrame

## Calculation

Calculate Volume Ratio

In [ ]:
for row in potentialDataFrame.index:
    potentialDataFrame.loc[row, 'T1 Volume Ratio'] = potentialDataFrame.loc[row, 'T1 Volume'] / potentialDataFrame.loc[row, 'T0 Volume']
    
potentialDataFrame

## Screening

Screening criteria are as follow:

- Average Volume can't be too low
- T1 Volume can't be too low
- T1 Volume is more than 5 times of T0

In [ ]:
volumeLowerBound = float(2e6)
t1traget = float(5)

potentialDataFrame = potentialDataFrame[
#     (volumeLowerBound < potentialDataFrame['30 day Average Volume']) &
#     (volumeLowerBound < potentialDataFrame['T1 Volume']) &
    (t1traget < potentialDataFrame['T1 Volume Ratio'])
]
potentialDataFrame.sort_values(
    'Ticker',
    ascending = True,
    inplace = True
)
potentialDataFrame.reset_index(inplace = True, drop = True)

potentialDataFrame

In [ ]:
','.join(potentialDataFrame['Ticker'].to_list())

## Data Export

For the ease of the next programe, we will also export the file in CSV version.

In [ ]:
from datetime import date
import os

today = date.today().strftime("%Y%m%d")
folderName = f'../export/{today}'
fileName = f'{folderName}/yau4muk6man4_strategy_US_{today}_volume_rise'
csvFileName = f'{fileName}.csv'
xlsxFileName = f'{fileName}.xlsx'
sheetName = f'{today} Volume Rise'

In [ ]:
try:
    os.mkdir(folderName)
    print("Directory " , folderName ,  " Created ") 
except FileExistsError:
    print("Directory " , folderName ,  " already exists")

Export to csv

In [ ]:
potentialDataFrame.to_csv(csvFileName, index = False)

Export to xlsx

In [ ]:
writer = pd.ExcelWriter(xlsxFileName, engine = 'xlsxwriter')
potentialDataFrame.to_excel(writer, sheet_name = sheetName, index = False)

In [ ]:
backgroundColor = '#0a0a23'
fontColor = '#ffffff'

stringTemplate = writer.book.add_format(
        {
            'font_color': fontColor,
            'bg_color': backgroundColor,
            'border': 1
        }
    )

dollarTemplate = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': fontColor,
            'bg_color': backgroundColor,
            'border': 1
        }
    )

integerTemplate = writer.book.add_format(
        {
            'num_format':'#,###',
            'font_color': fontColor,
            'bg_color': backgroundColor,
            'border': 1
        }
    )

floatTemplate = writer.book.add_format(
        {
            'num_format':'0.0',
            'font_color': fontColor,
            'bg_color': backgroundColor,
            'border': 1
        }
    )

percentTemplate = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': fontColor,
            'bg_color': backgroundColor,
            'border': 1
        }
    )

In [ ]:
columnFormats = {
    'A': ['Ticker', stringTemplate],
    'B': ['Price', dollarTemplate],
    'C': ['30 day Average Volume', integerTemplate],
    'D': ['T0 Date', stringTemplate],
    'E': ['T0 Volume', integerTemplate],
    'F': ['T1 Volume', integerTemplate],
    'G': ['T1 Volume Ratio', percentTemplate],
}

for column in columnFormats.keys():
    writer.sheets[sheetName].set_column(
        f'{column}:{column}',
        max(len(columnFormats[column][0]), 10),
        columnFormats[column][1]
    )
    writer.sheets[sheetName].write(
        f'{column}1',
        columnFormats[column][0],
        columnFormats[column][1]
    )

In [ ]:
writer.save()

## Telegram

In [ ]:
import telegram
from secrets import TELEGRAM_CHAT_ID
from secrets import TELEGRAM_API_TOKEN

In [ ]:
telegramBot = telegram.Bot(TELEGRAM_API_TOKEN)
telegramBot

In [ ]:
txt = f'''
⬆️⬆️⬆️ {today} Volume Rise ⬆️⬆️⬆️

This is report of stock that have rise in volume during {today}.
We have found {len(potentialDataFrame.index)} stock(s) that match the screening requirements.

Rise Ticker:
{','.join(potentialDataFrame['Ticker'])}

Detail as in follow xlsx.
'''

output = open(xlsxFileName, 'rb')

In [ ]:
if SEND_TELEGRAM:
    telegramBot.send_message(TELEGRAM_CHAT_ID, txt)
    telegramBot.send_document(TELEGRAM_CHAT_ID, output)